In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install vncorenlp
!wget -qO vncorenlp.zip https://github.com/vncorenlp/VnCoreNLP/archive/master.zip
!unzip -q vncorenlp.zip
!mv VnCoreNLP-master VnCoreNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=4919556498fd212b3cc6f45e764d4e9079bc69788e3a697d0592456638134f0c
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [4]:
import pandas as pd

In [6]:
path_viSL_dict = '/content/drive/MyDrive/Data_paper_4/output.xlsx'
path_data_test = '/content/drive/MyDrive/Data_paper_4/dataTest.txt'


In [7]:
df = pd.read_excel(path_viSL_dict)
set_words = set(df.iloc[:,1])

In [8]:
len(set_words)

4332

In [11]:
dict_viSL = dict()
for i, w in enumerate(set_words):
  dict_viSL[w] = i+1

In [10]:
import regex as re
import io

# List of Unicode characters with accents
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
# Corresponding characters without accents
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

# Function to load the dictionary for mapping accented to unaccented characters
def load_accent_dict():
    accent_dict = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
    for i in range(len(char1252)):
        accent_dict[char1252[i]] = charutf8[i]
    return accent_dict

accent_dict = load_accent_dict()

# Function to convert precomposed Unicode characters to decomposed form
def convert_unicode(text):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: accent_dict[x.group()], text
    )

# Vietnamese vowel table with tone marks and corresponding no-tone vowels
vowel_table = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
               ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
               ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
               ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
               ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
               ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
               ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
               ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
               ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
               ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
               ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
               ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

# Tone mark table
tone_mark_table = ['', 'f', 's', 'r', 'x', 'j']

# Mapping from Vietnamese vowels to their indices in the vowel table
vowel_to_ids = {}

for i in range(len(vowel_table)):
    for j in range(len(vowel_table[i]) - 1):
        vowel_to_ids[vowel_table[i][j]] = (i, j)

# Function to normalize the position of tone marks in a Vietnamese word
def normalize_vietnamese_word(word):
    if not is_valid_vietnamese_word(word):
        return word
    chars = list(word)
    tone_mark = 0
    vowel_indices = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = vowel_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check 'qu'
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check 'gi'
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            tone_mark = y
            chars[index] = vowel_table[x][0]
        if not qu_or_gi or index != 1:
            vowel_indices.append(index)

    if len(vowel_indices) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowel_to_ids.get(chars[1])
                chars[1] = vowel_table[x][tone_mark]
            else:
                x, y = vowel_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = vowel_table[x][tone_mark]
                else:
                    chars[1] = vowel_table[5][tone_mark] if chars[1] == 'i' else vowel_table[9][tone_mark]
            return ''.join(chars)
        return word

    for index in vowel_indices:
        x, y = vowel_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowel_table[x][tone_mark]
            return ''.join(chars)

    if len(vowel_indices) == 2:
        if vowel_indices[-1] == len(chars) - 1:
            x, y = vowel_to_ids[chars[vowel_indices[0]]]
            chars[vowel_indices[0]] = vowel_table[x][tone_mark]
        else:
            x, y = vowel_to_ids[chars[vowel_indices[1]]]
            chars[vowel_indices[1]] = vowel_table[x][tone_mark]
    else:
        x, y = vowel_to_ids[chars[vowel_indices[1]]]
        chars[vowel_indices[1]] = vowel_table[x][tone_mark]

    return ''.join(chars)

# Function to check if a word is a valid Vietnamese word
def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_index = -1
    for index, char in enumerate(chars):
        x, y = vowel_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_index == -1:
                vowel_index = index
            else:
                if index - vowel_index != 1:
                    return False
                vowel_index = index
    return True

# Function to normalize tone marks in a Vietnamese sentence
def normalize_vietnamese_sentence(sentence):
    """
    Normalize a Vietnamese sentence to standard tone mark positions.
    :param sentence: Input sentence in Vietnamese
    :return: Normalized sentence
    """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cleaned_word = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cleaned_word) == 3:
            cleaned_word[1] = normalize_vietnamese_word(cleaned_word[1])
        words[index] = ''.join(cleaned_word)
    return ' '.join(words)


In [22]:
from vncorenlp import VnCoreNLP
vncorenlp_path = "/content/VnCoreNLP/VnCoreNLP-1.2.jar"

rdrsegmenter = VnCoreNLP(vncorenlp_path, annotators="wseg", max_heap_size='-Xmx2g')

def process_sentence(sentence):
    lw = sentence.split()
    return [w.lower() for w in lw if len(w)>1 or w.isdigit()]

def process_text_file(input_file, rdrsegmenter):
    """Xử lý văn bản từ file input, loại bỏ ký tự đặc biệt đầu dòng và dấu câu."""
    processed_lines = []

    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    for line in lines:
        line = normalize_vietnamese_sentence(line)
        line = re.sub(r'^[^\w\d]+', '', line.strip())
        sentences = re.split(r'[.!?]', line)
        for sentence in sentences:
            cleaned_sentence = re.sub(r'(?<!\S)[^\w\s-\/](?!\S)', '', sentence).strip()
            if cleaned_sentence and len(cleaned_sentence) > 2:
                tokenized_sentences = rdrsegmenter.tokenize(cleaned_sentence)
                for tokenized_sentence in tokenized_sentences:
                    processed_lines.append(process_sentence(' '.join(tokenized_sentence)))
    return processed_lines

In [23]:
list_sentences = process_text_file(path_data_test, rdrsegmenter)

In [24]:
len(list_sentences)

164

In [25]:
list_sentences[:5]

[['bộ',
  'y_tế',
  'đã',
  'có',
  'quyết_định',
  '2609',
  'qđ-byt',
  'ban_hành',
  'hướng_dẫn',
  'phòng',
  'và',
  'kiểm_soát',
  'lây_nhiễm',
  'covid-19',
  'trong',
  'cơ_sở',
  'khám',
  'bệnh',
  'chữa',
  'bệnh'],
 ['hướng_dẫn',
  'phòng',
  'và',
  'kiểm_soát',
  'lây_nhiễm',
  'covid-19',
  'trong',
  'cơ_sở',
  'khám',
  'bệnh',
  'chữa',
  'bệnh',
  'nêu',
  'rõ',
  'các',
  'biện_pháp',
  'kiểm_soát',
  'lây_nhiễm',
  'sars-cov',
  '-2',
  'trong',
  'cơ_sở',
  'khám',
  'chữa',
  'bệnh',
  'như',
  'sau'],
 ['tổ_chức',
  'phát_hiện',
  'sớm',
  'và',
  'cách_ly',
  'kịp_thời',
  'người',
  'mắc',
  'hoặc',
  'nghi',
  'mắc',
  'covid-19'],
 ['người_bệnh',
  'có',
  'dấu_hiệu',
  'chỉ_điểm',
  'mắc',
  'covid-19',
  'ho',
  'sốt',
  'khi',
  'đến',
  'cơ_sở',
  'khám',
  'bệnh',
  'chữa',
  'bệnh',
  'được',
  'chỉ_dẫn',
  'khám',
  'tại',
  'phòng_khám',
  'truyền_nhiễm',
  'người_bệnh',
  'cấp_cứu',
  'được',
  'chỉ_dẫn',
  'khám',
  'tại',
  'khu_vực',
  'tiếp_đón'

In [26]:
def encode_sentences(sentences, dict_viSL):
    encoded_sentences = []
    total_words = 0  # Tổng số từ
    total_zero_encoded = 0  # Tổng số từ được mã hóa thành 0

    for sentence in sentences:
        encoded_sentence = []
        for word in sentence:
            total_words += 1  # Đếm tổng số từ
            if word.isdigit():  # Kiểm tra xem từ có phải là số không
                encoded_sentence.append("Num")
            elif word in dict_viSL:  # Kiểm tra từ trong từ điển
                encoded_sentence.append(dict_viSL[word])
            else:
                encoded_sentence.append(0)
                total_zero_encoded += 1  # Đếm số từ được mã hóa thành 0
        encoded_sentences.append(encoded_sentence)

    return encoded_sentences, total_words, total_zero_encoded

In [28]:
encoded_result, total_words, total_zero_encoded = encode_sentences(list_sentences, dict_viSL)

print(f"Encryption result: {encoded_result}")
print(f"Total words: {total_words}")
print(f"Total words encoded as 0: {total_zero_encoded}")

Encryption result: [[0, 0, 0, 783, 498, 'Num', 0, 0, 2446, 2635, 1300, 0, 1638, 0, 3944, 0, 2499, 2682, 0, 2682], [2446, 2635, 1300, 0, 1638, 0, 3944, 0, 2499, 2682, 0, 2682, 0, 1791, 0, 470, 0, 1638, 0, 0, 3944, 0, 2499, 0, 2682, 0, 0], [2264, 0, 3534, 1300, 2745, 0, 942, 2267, 0, 0, 2267, 0], [0, 783, 2882, 0, 2267, 0, 2561, 854, 2040, 166, 0, 2499, 2682, 0, 2682, 3334, 0, 2499, 0, 0, 0, 0, 3949, 3334, 0, 2499, 0, 0, 0, 0, 0, 3949], [0, 2505, 120, 4018, 0], [0, 0, 0, 0, 437, 1815, 3334, 2745, 1348, 0, 0, 3949, 3485, 0, 3944, 0, 3949, 0, 3334, 2745, 1348, 0, 0], [0, 0, 0, 0, 3485, 0, 0, 2267, 0, 0, 0, 0, 0, 0, 2745, 0, 0, 0, 2745, 1348], [0, 2499, 2682, 0, 1679, 0, 0, 0, 3949, 1300, 0, 0, 2745, 0, 0, 0, 0, 437, 0, 1242, 0, 0, 0, 1300, 3532, 783, 0, 0, 3597, 1969, 2745, 0, 0], [4180, 0, 1496, 0, 942, 2267, 0, 0, 2267, 0, 474, 0, 1067, 0, 1300, 524, 0, 0, 0, 3041, 2316], [0, 0, 470, 0, 0, 0, 0, 0, 0, 3792, 0], [0, 0, 0, 0, 0, 0, 0, 3792, 3332, 3792, 0, 1300, 3792, 0, 2547, 0, 0, 0, 0, 0

In [30]:
import os
import csv
POS_TAG_ABBREVIATIONS = {
    'adj': 'A',
    'noun': 'N',
    'verb': 'V',
    'adv': 'ADV'
}

CATEGORY_ABBREVIATIONS = {
    'color': 'C',
    'size': 'S',
    'motion': 'M',
    'contact': 'CT',
    'state': 'ST',
    'artifact': 'ART',
    'body': 'BD',
    'group': 'GR',
    'communication': 'COM',
    'perception': 'PERC',
    'possession': 'POS',
    'social': 'SOC',
    'stative': 'STV'
}
# Lớp Category để chứa danh sách từ đồng nghĩa
class Category:
    def __init__(self, name):
        self.name = name
        self.synonyms = []

    def add_synonyms(self, synonyms):
        self.synonyms.append(synonyms)

    def get_synonyms(self):
        return self.synonyms

    def __repr__(self):
        return f"Category(name={self.name}, synonyms={self.synonyms})"

# Lớp POSTag để chứa nhãn từ loại và danh mục
class POSTag:
    def __init__(self, pos_name):
        self.pos_name = pos_name
        self.categories = {}  # Từ điển {category_name: Category}
        self.synonyms = []    # Lưu những từ đồng nghĩa nếu không có danh mục

    def add_category(self, category_name):
        if category_name not in self.categories:
            self.categories[category_name] = Category(category_name)

    def add_synonyms_to_category(self, category_name, synonyms):
        if category_name in self.categories:
            self.categories[category_name].add_synonyms(synonyms)

    def add_synonyms(self, synonyms):
        self.synonyms.append(synonyms)

    def get_all_synonyms(self):
        # Lấy tất cả từ đồng nghĩa từ cả danh mục và không danh mục
        all_synonyms = self.synonyms[:]
        for category in self.categories.values():
            all_synonyms.extend(category.get_synonyms())
        return all_synonyms

    def get_synonyms_by_category(self, category_name):
        if category_name in self.categories:
            return self.categories[category_name].get_synonyms()
        return []

    def __repr__(self):
        return f"POSTag(pos_name={self.pos_name}, categories={self.categories}, synonyms={self.synonyms})"

class Words:
    def __init__(self):
        self.pos_tags = {}  # Từ điển {pos_tag_name: POSTag}

    def add_pos_tag(self, pos_tag_name):
        if pos_tag_name not in self.pos_tags:
            self.pos_tags[pos_tag_name] = POSTag(pos_tag_name)

    def add_category_to_pos_tag(self, pos_tag_name, category_name):
        if pos_tag_name in self.pos_tags:
            self.pos_tags[pos_tag_name].add_category(category_name)

    def add_synonyms_to_category(self, pos_tag_name, category_name, synonyms):
        if pos_tag_name in self.pos_tags:
            self.pos_tags[pos_tag_name].add_synonyms_to_category(category_name, synonyms)

    def add_synonyms_to_pos_tag(self, pos_tag_name, synonyms):
        if pos_tag_name in self.pos_tags:
            self.pos_tags[pos_tag_name].add_synonyms(synonyms)

    def get_synonyms_by_pos_tag(self, pos_tag_name):
        if pos_tag_name in self.pos_tags:
            return self.pos_tags[pos_tag_name].get_all_synonyms()
        return []

    def get_synonyms_by_category(self, pos_tag_name, category_name):
        if pos_tag_name in self.pos_tags:
            return self.pos_tags[pos_tag_name].get_synonyms_by_category(category_name)
        return []

    # Phương thức tìm kiếm thông tin của một từ
    def find_word(self, word):
        results = []
        word = word.replace(' ', '_')  # Đảm bảo từ cần tìm có dạng đúng (_ thay vì khoảng trắng)
        for pos_tag_name, pos_tag in self.pos_tags.items():
            # Kiểm tra từ trong danh sách từ đồng nghĩa không có danh mục
            for synonym_group in pos_tag.synonyms:
                if word in synonym_group:
                    results.append(f"Từ '{word}' thuộc POS-TAG '{pos_tag_name}' và không có danh mục.")

            # Kiểm tra từ trong các danh mục
            for category_name, category in pos_tag.categories.items():
                for synonym_group in category.get_synonyms():
                    if word in synonym_group:
                        results.append(f"Từ '{word}' thuộc POS-TAG '{pos_tag_name}' trong danh mục '{category_name}'.")

        if not results:
            return f"Từ '{word}' không tìm thấy."
        return "\n".join(results)

    # Phương thức tạo từ điển từ - {pos-tag, category} với ký hiệu ngắn gọn
    def generate_word_dict(self):
        word_dict = {}

        # Duyệt qua tất cả các POS-TAG
        for pos_tag_name, pos_tag in self.pos_tags.items():
            # Duyệt qua từ đồng nghĩa không có danh mục
            for synonym_group in pos_tag.synonyms:
                for word in synonym_group:
                    if word not in word_dict:
                        word_dict[word] = []
                    abbrev_pos_tag = POS_TAG_ABBREVIATIONS.get(pos_tag_name, pos_tag_name)
                    word_dict[word].append({'pos_tag': abbrev_pos_tag, 'category': None})

            # Duyệt qua từ đồng nghĩa có danh mục
            for category_name, category in pos_tag.categories.items():
                for synonym_group in category.get_synonyms():
                    for word in synonym_group:
                        if word not in word_dict:
                            word_dict[word] = []
                        abbrev_pos_tag = POS_TAG_ABBREVIATIONS.get(pos_tag_name, pos_tag_name)
                        abbrev_category = CATEGORY_ABBREVIATIONS.get(category_name, category_name)
                        word_dict[word].append({'pos_tag': abbrev_pos_tag, 'category': abbrev_category})

        return word_dict
    def generate_numbered_synonyms_dict(self):
        numbered_synonyms_dict = {}
        current_key = 1

        # Duyệt qua tất cả các POS-TAG
        for pos_tag_name, pos_tag in self.pos_tags.items():
            # Xử lý các từ đồng nghĩa không có danh mục
            for synonym_group in pos_tag.synonyms:
                numbered_synonyms_dict[current_key] = {
                    'synonyms': synonym_group,
                    'info': [pos_tag_name, None]  # Không có danh mục
                }
                current_key += 1

            # Xử lý các từ đồng nghĩa có danh mục
            for category_name, category in pos_tag.categories.items():
                for synonym_group in category.get_synonyms():
                    numbered_synonyms_dict[current_key] = {
                        'synonyms': synonym_group,
                        'info': [pos_tag_name, category_name]
                    }
                    current_key += 1

        return numbered_synonyms_dict
    def find_keys_by_word(self, word, numbered_synonyms_dict):
        matching_keys = []
        word = word.replace(' ', '_')  # Đảm bảo từ có định dạng dùng dấu gạch dưới

        # Duyệt qua từ điển và tìm các key mà danh sách từ đồng nghĩa chứa từ cần tìm
        for key, value in numbered_synonyms_dict.items():
            if word in value['synonyms']:
                matching_keys.append(key)

        if not matching_keys:
            return f"Từ '{word}' không xuất hiện trong danh sách."
        return matching_keys
    def generate_synonyms_dict(self):
        synonyms_dict = {}

        # Duyệt qua tất cả các POS-TAG
        for pos_tag_name, pos_tag in self.pos_tags.items():
            # Xử lý các từ đồng nghĩa không có danh mục
            for synonym_group in pos_tag.synonyms:
                if tuple(synonym_group) not in synonyms_dict:
                    synonyms_dict[tuple(synonym_group)] = []
                synonyms_dict[tuple(synonym_group)].append({
                    'pos_tag': pos_tag_name,
                    'category': None
                })

            # Xử lý các từ đồng nghĩa có danh mục
            for category_name, category in pos_tag.categories.items():
                for synonym_group in category.get_synonyms():
                    if tuple(synonym_group) not in synonyms_dict:
                        synonyms_dict[tuple(synonym_group)] = []
                    synonyms_dict[tuple(synonym_group)].append({
                        'pos_tag': pos_tag_name,
                        'category': category_name
                    })

        return synonyms_dict
    # Phương thức liệt kê tất cả các từ trong class Words
    def list_all_words(self):
        all_words = set()

        # Duyệt qua tất cả các POS-TAG
        for pos_tag_name, pos_tag in self.pos_tags.items():
            for synonym_group in pos_tag.synonyms:
                all_words.update(synonym_group)

            for category_name, category in pos_tag.categories.items():
                for synonym_group in category.get_synonyms():
                    all_words.update(synonym_group)

        return all_words

    def __repr__(self):
        return f"Words(pos_tags={self.pos_tags})"

def process_file(file_path, words_data, setWords):
    file_name = os.path.basename(file_path)
    name_parts = file_name.split('.')

    if len(name_parts) == 3:
        pos_tag = name_parts[0]
        category = name_parts[1]

        words_data.add_pos_tag(pos_tag)
        words_data.add_category_to_pos_tag(pos_tag, category)

        with open(file_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                synonyms = [word.strip().replace(' ', '_').lower() for word in row if word.strip()]
                if any(word in setWords for word in synonyms):
                    words_data.add_synonyms_to_category(pos_tag, category, synonyms)

    elif len(name_parts) == 2:
        pos_tag = name_parts[0]

        words_data.add_pos_tag(pos_tag)

        with open(file_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                synonyms = [word.strip().replace(' ', '_').lower() for word in row if word.strip()]
                if any(word in setWords for word in synonyms):
                    words_data.add_synonyms_to_pos_tag(pos_tag, synonyms)

def process_directory(directory_path, setWords):
    words_data = Words()

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            process_file(file_path, words_data, setWords)

    return words_data
directory_path = '/content/drive/MyDrive/Data_paper_4/Dict'
words_data = process_directory(directory_path, set_words)
#print(words_data)

In [31]:
# Giả sử bạn đã thêm dữ liệu vào words_data
all_words = words_data.list_all_words()

# In ra tất cả các từ
print("Tất cả các từ trong Words:")
c = 0
for word in all_words:
    if c< 10:

        print(word)
        c+=1

Tất cả các từ trong Words:
cú_đấm
cảnh_nghèo
gian_khổ
cản_phá
suy_ngẫm
gảy_đàn
ngất_xỉu
đáng_tin
khí_đốt
yêu_quý


In [32]:
len(all_words)

16627

In [33]:
word_dict = words_data.generate_word_dict()
'''for word, info in word_dict.items():
    print(f"Từ: {word}")
    for entry in info:
        pos_tag = entry['pos_tag']
        category = entry['category'] if entry['category'] else 'None'
        print(f"  - POS-TAG: {pos_tag}, Category: {category}")'''

'for word, info in word_dict.items():\n    print(f"Từ: {word}")\n    for entry in info:\n        pos_tag = entry[\'pos_tag\']\n        category = entry[\'category\'] if entry[\'category\'] else \'None\'\n        print(f"  - POS-TAG: {pos_tag}, Category: {category}")'

In [34]:
# Giả sử words_data đã được xử lý từ các file CSV
synonyms_dict = words_data.generate_synonyms_dict()

# In ra từ điển
'''for synonyms, info in synonyms_dict.items():
    print(f"Từ đồng nghĩa: {synonyms}")
    for entry in info:
        pos_tag = entry['pos_tag']
        category = entry['category'] if entry['category'] else 'Không có danh mục'
        print(f"  - POS-TAG: {pos_tag}, Category: {category}")'''

'for synonyms, info in synonyms_dict.items():\n    print(f"Từ đồng nghĩa: {synonyms}")\n    for entry in info:\n        pos_tag = entry[\'pos_tag\']\n        category = entry[\'category\'] if entry[\'category\'] else \'Không có danh mục\'\n        print(f"  - POS-TAG: {pos_tag}, Category: {category}")'

In [35]:
len(synonyms_dict)

8812

In [36]:
# Giả sử words_data đã được xử lý từ các file CSV
numbered_dict = words_data.generate_numbered_synonyms_dict()

# In ra từ điển
'''for key, value in numbered_dict.items():
    synonyms = value['synonyms']
    pos_tag = value['info'][0]
    category = value['info'][1] if value['info'][1] else 'Không có danh mục'
    print(f"Key {key}: {synonyms}, POS-TAG: {pos_tag}, Category: {category}")'''

'for key, value in numbered_dict.items():\n    synonyms = value[\'synonyms\']\n    pos_tag = value[\'info\'][0]\n    category = value[\'info\'][1] if value[\'info\'][1] else \'Không có danh mục\'\n    print(f"Key {key}: {synonyms}, POS-TAG: {pos_tag}, Category: {category}")'

In [37]:
print(len(numbered_dict))

9521


In [39]:
# Find keys that contain the word 'brilliant' in the synonym list
word_to_find = 'rực_rỡ'
matching_keys = words_data.find_keys_by_word(word_to_find, numbered_dict)

print(f"Keys containing words'{word_to_find}': {matching_keys}")

Keys containing words'rực_rỡ': [7276, 7397, 7438, 7441, 7878, 7971, 8006, 8155, 9086, 9481]


In [40]:
for key in matching_keys:
  print (numbered_dict[key])

{'synonyms': ['rực_rỡ', 'đa_sắc'], 'info': ['adj', None]}
{'synonyms': ['nguy_nga', 'tráng_lệ', 'lộng_lẫy', 'huy_hoàng', 'rực_rỡ', 'hoa_lệ'], 'info': ['adj', None]}
{'synonyms': ['rực_rỡ', 'chói_lọi', 'sặc_sỡ', 'loè_loẹt'], 'info': ['adj', None]}
{'synonyms': ['rực', 'chói_lọi', 'sặc_sỡ', 'rực_rỡ', 'loè_loẹt', 'màu_mè'], 'info': ['adj', None]}
{'synonyms': ['sáng_chói', 'sáng_ngời', 'rực_rỡ', 'huy_hoàng', 'chói_lọi'], 'info': ['adj', None]}
{'synonyms': ['sáng', 'rực_rỡ', 'sáng_chói', 'sáng_ngời', 'chói_lọi', 'huy_hoàng', 'lộng_lẫy'], 'info': ['adj', None]}
{'synonyms': ['lộng_lẫy', 'rực_rỡ'], 'info': ['adj', None]}
{'synonyms': ['sáng_rực', 'rực_rỡ'], 'info': ['adj', None]}
{'synonyms': ['rực_rỡ', 'huy_hoàng', 'tráng_lệ', 'hoa_lệ', 'lộng_lẫy'], 'info': ['adj', None]}
{'synonyms': ['rực_rỡ'], 'info': ['adv', None]}


In [59]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def load_phobert_model():
   # Load the tokenizer and the PhoBERT model
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    model = AutoModel.from_pretrained("vinai/phobert-base")
    return tokenizer, model

# Function to calculate the embedding of a sentence
def get_sentence_embedding(sentence, tokenizer, model):
    try:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)

        if outputs.last_hidden_state.size(1) == 0:
            print(f"Warning: No tokens produced for the sentence: '{sentence}'")
            return None

        # Get the embedding of the [CLS] token to represent the embedding of the whole sentence
        cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
        return cls_embedding

    except IndexError as e:
        print(f"IndexError encountered: {e} - Skipping this sentence: '{sentence}'")
        return None  # Hoặc bạn có thể trả về một embedding mặc định như np.zeros((768,)) nếu cần


# Function to calculate cosine similarity between two embeddings
def cosine_similarity_score(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]

def find_best_key_by_context(sentence, index_to_replace, word_to_replace, keys, numbered_synonyms_dict, tokenizer, model):
    if len(keys) == 1:
        return keys[0]
    else:
        original_sentence = " ".join(sentence)
        original_embedding = get_sentence_embedding(original_sentence, tokenizer, model)

        if original_embedding is None:
            return None

        best_key = None
        best_similarity = -1

        for key in keys:
            synonyms = numbered_synonyms_dict[key]['synonyms']

            for synonym in synonyms:
                if synonym == word_to_replace:
                    continue

                replaced_sentence = sentence.copy()
                replaced_sentence[index_to_replace] = synonym
                replaced_sentence_str = " ".join(replaced_sentence)

                replaced_embedding = get_sentence_embedding(replaced_sentence_str, tokenizer, model)

                if replaced_embedding is None:
                    continue

                similarity = cosine_similarity_score(original_embedding, replaced_embedding)

                if similarity > best_similarity:
                    best_similarity = similarity
                    best_key = key

        return best_key

tokenizer, model = load_phobert_model()


In [ ]:
# Giả sử chúng ta có câu input và các key tương ứng chứa từ đồng nghĩa
#sentence_1 = ["ánh", "mặt_trời", "sáng", "rực_rỡ", "xua", "tan", "đi", "làn_mây", "đen", "tối"]
sentence = ["ánh", "lửa", "bập_bùng", "cháy", "rực_rỡ", "trong","buổi", "đốt","lửa","trại", "của", "đêm", "giao_thừa"]
index_to_replace = 4  # Vị trí của từ 'rực_rỡ'
word_to_replace = sentence[index_to_replace]

# Danh sách các key chứa từ đồng nghĩa với 'rực_rỡ' (giả sử đã tìm được)
keys = [1, 2, 3 , 4]

# Từ điển numbered_synonyms_dict giả định
numbered_synonyms_dict = {

    1: {'synonyms': ['rực_rỡ', 'huy_hoàng', 'tráng_lệ', 'hoa_lệ', 'lộng_lẫy', 'nguy_nga' ], 'info': ['adj', None]},
    2: {'synonyms': ['rực_rỡ', 'chói_lọi', 'sặc_sỡ', 'loè_loẹt'], 'info': ['adj', None]},
    3: {'synonyms': ['sáng_chói', 'sáng_ngời', 'rực_rỡ', 'huy_hoàng', 'chói_lọi'], 'info': ['adj', None]},
    4: {'synonyms': ['rực_rỡ', 'đa_sắc' ], 'info': ['adj', None]}
}

# Tải mô hình PhoBERT


# Tìm key phù hợp nhất dựa trên ngữ cảnh
best_key , best_similarity = find_best_key_by_context(sentence, index_to_replace, word_to_replace, keys, numbered_synonyms_dict, tokenizer, model)

# In ra kết quả
print(f"Key phù hợp nhất: {best_key}  best_similarity {best_similarity}")

In [ ]:
import time
start_time = time.time()
best_key = find_best_key_by_context(sentence, index_to_replace, word_to_replace, keys, numbered_synonyms_dict, tokenizer, model)
print(f"Key phù hợp nhất: {best_key} time = {time.time() - start_time}")


Key phù hợp nhất: 168 time =0.9431040287017822


In [56]:
def encoding_sentence(sentence):
    result = []
    for index, word in enumerate(sentence):
        if word not in all_words:
            if word.isdigit():
              result.append("Num")
            else:
              result.append(0)
        else:
            keys =  words_data.find_keys_by_word(word, numbered_dict)
            best_key = find_best_key_by_context(sentence, index, word, keys, numbered_dict, tokenizer, model)
            result.append(best_key)
    return result

In [54]:
list_sentences[0]

['bộ',
 'y_tế',
 'đã',
 'có',
 'quyết_định',
 '2609',
 'qđ-byt',
 'ban_hành',
 'hướng_dẫn',
 'phòng',
 'và',
 'kiểm_soát',
 'lây_nhiễm',
 'covid-19',
 'trong',
 'cơ_sở',
 'khám',
 'bệnh',
 'chữa',
 'bệnh']

In [57]:
import time
start_time = time.time()
encoding = encoding_sentence(list_sentences[0])
print(f"encoding: {encoding} time = {time.time() - start_time}")

encoding: [5720, 0, 7888, 2928, 4559, 'Num', 0, 1872, 1608, 1025, 9519, 941, 4082, 0, 8111, 4963, 580, 3949, 3090, 3949] time = 21.98444151878357


In [58]:
encoding_res = []
count_zero = 0
for i in range (len(list_sentences)):
  encoding = encoding_sentence(list_sentences[0])
  count_zero += encoding.count(0)
  encoding_res.append(encoding)

print(f"Total words encoded as 0: {count_zero}")

Total words encoded as 0: 492
